In [1]:
from PIL import Image
from os import listdir
from os.path import isdir
from numpy import asarray, expand_dims
import numpy as np

In [2]:
def load_face(filename):
    
    image = Image.open(filename)

    image = image.convert('RGB')
    
    return asarray(image)    

In [3]:
def load_faces(directory_src):
    
    faces = list()
    
    for filename in listdir(directory_src):
        
        path = directory_src + filename
        
        try:
            faces.append(load_face(path))
        except:
            print("Erro na imagem {}".format(path))
    
    return faces

In [4]:
def load_fotos(directory_src):
    
    X, y = list(), list()
    
    for subdir in listdir (directory_src):
        
        path = directory_src + subdir + '\\'
        
        if not isdir(path):
            continue
        
        faces = load_faces(path)
        
        labels = [subdir for _ in range(len(faces))]
        
        ## sumarizando o progresso
        
        print('>Carregadas %d faces da classe: %s' % (len(faces),subdir))
        
        X.extend(faces)
        y.extend(labels)
        
    return asarray(X), asarray(y)

# Carregando todas as Imagens

In [5]:
trainX, trainy = load_fotos(directory_src = "D:\\Dataset\\Faces\\train\\")

>Carregadas 316 faces da classe: Amanda
>Carregadas 34 faces da classe: Anita


In [6]:
trainX.shape

(350, 160, 160, 3)

In [7]:
trainy.shape

(350,)

In [8]:
from tensorflow.keras.models import load_model

In [9]:
model = load_model('facenet_keras.h5')

In [10]:
model.summary()

Model: "inception_resnet_v1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 160, 160, 3) 0                                            
__________________________________________________________________________________________________
Conv2d_1a_3x3 (Conv2D)          (None, 79, 79, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
Conv2d_1a_3x3_BatchNorm (BatchN (None, 79, 79, 32)   96          Conv2d_1a_3x3[0][0]              
__________________________________________________________________________________________________
Conv2d_1a_3x3_Activation (Activ (None, 79, 79, 32)   0           Conv2d_1a_3x3_BatchNorm[0][0]    
________________________________________________________________________________

Block35_5_Branch_2_Conv2d_0a_1x (None, 17, 17, 32)   8192        Block35_4_Activation[0][0]       
__________________________________________________________________________________________________
Block35_5_Branch_2_Conv2d_0a_1x (None, 17, 17, 32)   96          Block35_5_Branch_2_Conv2d_0a_1x1[
__________________________________________________________________________________________________
Block35_5_Branch_2_Conv2d_0a_1x (None, 17, 17, 32)   0           Block35_5_Branch_2_Conv2d_0a_1x1_
__________________________________________________________________________________________________
Block35_5_Branch_1_Conv2d_0a_1x (None, 17, 17, 32)   8192        Block35_4_Activation[0][0]       
__________________________________________________________________________________________________
Block35_5_Branch_2_Conv2d_0b_3x (None, 17, 17, 32)   9216        Block35_5_Branch_2_Conv2d_0a_1x1_
__________________________________________________________________________________________________
Block35_5_

# FUNÇÃO GERADORA DE EMBEDDINS

In [11]:
def get_embedding(model, face_pixels):
    
    ## Padronização
    mean, std = face_pixels.mean(), face_pixels.std()
    face_pixels = (face_pixels - mean)/std
    
    ##Transformar a face em um único exemplo
    
    samples = expand_dims(face_pixels, axis=0)
    
    ## Realizar a predição gerando embeddins
    
    yhat = model.predict(samples)
    
    return yhat[0]

In [12]:
newTrainX = list()

In [13]:
for face in trainX:
    embedding = get_embedding(model, face)
    newTrainX.append(embedding)

In [14]:
newTrainX = asarray(newTrainX)

In [15]:
newTrainX.shape

(350, 128)

In [16]:
import pandas as pd

In [17]:
df = pd.DataFrame(data=newTrainX)

In [18]:
df

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
0,-1.103358,-1.240901,0.277070,0.280350,-0.741683,-0.683142,0.446209,-3.006840,-0.580341,0.436692,...,-1.808459,-0.768865,1.423394,0.372536,1.049107,-2.107272,-0.159932,1.899448,1.410276,-0.510299
1,-0.077970,-0.730480,0.489072,-1.157417,0.215416,-0.335212,0.070179,-0.205632,-0.163127,-0.526533,...,1.084607,0.732919,0.077353,-0.223365,0.613588,0.051901,0.054338,-0.145959,0.605981,0.103310
2,-0.455212,-0.505452,0.328990,-0.977613,0.666581,-0.263076,0.291476,0.047229,0.019557,-0.416860,...,0.986564,0.605018,0.052781,-0.116526,0.481358,0.178251,0.070979,0.073808,0.650760,0.260512
3,-0.399531,-0.303500,0.017778,-1.863094,0.235948,0.444989,0.221935,0.664270,0.464147,-0.261184,...,0.706362,0.156992,0.088551,0.322296,0.442675,0.169842,1.090934,-0.281188,0.462268,-0.473904
4,-0.234686,-0.515040,0.314227,-0.987969,0.376778,0.017829,-0.234044,-0.110165,-0.003368,-0.135246,...,0.986988,0.566991,-0.209948,-0.293152,0.538615,0.155216,0.117159,0.053132,1.023518,0.251717
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,-0.044982,0.141337,0.669841,-1.055132,-0.554959,-0.094389,0.808895,0.614182,0.682296,0.018864,...,0.720815,-0.020778,1.401459,0.001832,-0.998806,-0.798887,0.550593,-0.330366,0.516867,-0.137979
346,-0.042547,-0.553033,0.398286,-0.168912,-0.801785,0.209301,0.224039,-0.109003,0.097656,-0.598394,...,1.262626,-0.876224,0.774390,-0.794122,0.191423,-0.405578,0.107491,0.157718,0.710793,1.050327
347,-0.111479,-0.695575,0.106272,-0.927812,0.374847,1.148341,-0.268901,0.298166,0.316038,0.019069,...,0.488520,-0.694429,-0.053003,0.800918,0.053875,-0.224970,0.533747,-0.135428,0.331979,0.782654
348,-0.286212,-0.381350,-0.078657,-0.151145,0.115800,0.271336,-0.105777,0.057974,0.275087,-0.147204,...,0.145897,0.317215,0.014155,-0.480993,-0.011739,0.706245,-0.050755,0.089455,0.571589,0.172153


In [19]:
df['target'] = trainy

In [20]:
df

,0,1,2,3,4,5,6,7,8,9,...,119,120,121,122,123,124,125,126,127,target
0,-1.103358,-1.240901,0.277070,0.280350,-0.741683,-0.683142,0.446209,-3.006840,-0.580341,0.436692,...,-0.768865,1.423394,0.372536,1.049107,-2.107272,-0.159932,1.899448,1.410276,-0.510299,Amanda
1,-0.077970,-0.730480,0.489072,-1.157417,0.215416,-0.335212,0.070179,-0.205632,-0.163127,-0.526533,...,0.732919,0.077353,-0.223365,0.613588,0.051901,0.054338,-0.145959,0.605981,0.103310,Amanda
2,-0.455212,-0.505452,0.328990,-0.977613,0.666581,-0.263076,0.291476,0.047229,0.019557,-0.416860,...,0.605018,0.052781,-0.116526,0.481358,0.178251,0.070979,0.073808,0.650760,0.260512,Amanda
3,-0.399531,-0.303500,0.017778,-1.863094,0.235948,0.444989,0.221935,0.664270,0.464147,-0.261184,...,0.156992,0.088551,0.322296,0.442675,0.169842,1.090934,-0.281188,0.462268,-0.473904,Amanda
4,-0.234686,-0.515040,0.314227,-0.987969,0.376778,0.017829,-0.234044,-0.110165,-0.003368,-0.135246,...,0.566991,-0.209948,-0.293152,0.538615,0.155216,0.117159,0.053132,1.023518,0.251717,Amanda
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,-0.044982,0.141337,0.669841,-1.055132,-0.554959,-0.094389,0.808895,0.614182,0.682296,0.018864,...,-0.020778,1.401459,0.001832,-0.998806,-0.798887,0.550593,-0.330366,0.516867,-0.137979,Anita
346,-0.042547,-0.553033,0.398286,-0.168912,-0.801785,0.209301,0.224039,-0.109003,0.097656,-0.598394,...,-0.876224,0.774390,-0.794122,0.191423,-0.405578,0.107491,0.157718,0.710793,1.050327,Anita
347,-0.111479,-0.695575,0.106272,-0.927812,0.374847,1.148341,-0.268901,0.298166,0.316038,0.019069,...,-0.694429,-0.053003,0.800918,0.053875,-0.224970,0.533747,-0.135428,0.331979,0.782654,Anita
348,-0.286212,-0.381350,-0.078657,-0.151145,0.115800,0.271336,-0.105777,0.057974,0.275087,-0.147204,...,0.317215,0.014155,-0.480993,-0.011739,0.706245,-0.050755,0.089455,0.571589,0.172153,Anita


In [21]:
df.to_csv('faces.csv', index=False)